In [209]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from scipy.integrate import quad
from scipy.special import hyp2f1

from StringIO import StringIO 
%matplotlib inline


In [203]:
data=pd.read_csv('mock_cluster.csv')
data.rename(columns={'Unnamed: 0': 'radius2D', '0':'shellId', '1':'x', '2':'y', '3':'z', '4':'v'},inplace=True)
data.radius2D = np.sqrt(np.power(data.x,2)+np.power(data.y,2))
data.describe()


,radius2D,shellId,x,y,z,v
count,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000
mean,0.749263,5.693159,-0.025881,-0.024694,-0.038884,52.580055
std,1.146764,2.078862,0.979960,0.956779,1.003943,741.213093
min,0.001035,0.000000,-5.908787,-4.680436,-7.796008,-3133.809266
25%,0.121634,4.000000,-0.211761,-0.218818,-0.170836,-273.311379
50%,0.354805,6.000000,-0.003869,-0.004193,-0.000696,14.626836
75%,0.877622,7.000000,0.144071,0.165231,0.156901,348.657303
max,8.971114,10.000000,7.653392,8.014847,8.520169,4572.619881


In [204]:
N_total =  data['x'].count()
del data['x']
del data['y']
del data['z']
del data['shellId']
#print N_total

In [205]:
# here created the grid cell boundaries for our entries 
def get_the_grid(arr):
    # 1D array
    # has to be sorted!!! 
    arr = np.array(arr)
    grid = (arr[1:]+arr[:-1])/2.
    #print grid.shape
    #print arr[:-1].shape
    grid = np.append(2.*arr[0]-grid[0], grid)
    grid = np.append(grid, 2.*arr[-1]-grid[-1])
    #grid = np.append(grid, 2.*arr[arr.shape[0]-1]-grid[-1])
    grid_low = grid[:-1]
    grid_up = grid[1:]
    return grid_low, grid_up                                                                                                                                                   

data = data.sort(['radius2D'])
grid_low, grid_up = get_the_grid(data['radius2D'])
if grid_low[0]<0:
    grid_low[0]=0.
#print grid_low.shape, grid_up.shape, data.radius2D.shape
data['dr_in']= grid_low
data['dr_out'] = grid_up

data = data.sort(['v'])
grid_low, grid_up = get_the_grid(data['v'])
data['dv_in']= grid_low
data['dv_out'] = grid_up


#data_vel_sort = data.sort(['v'])

In [206]:
print data

     radius2D            v     dr_in    dr_out        dv_in       dv_out
222  0.096502 -3133.809266  0.096449  0.096595 -3205.554701 -3062.063830
17   0.001035 -2990.318395  0.000576  0.001493 -3062.063830 -2904.178149
41   0.033606 -2818.037904  0.033436  0.033734 -2904.178149 -2796.931145
174  0.123581 -2775.824386  0.123571  0.123643 -2796.931145 -2763.845312
1    0.005904 -2751.866238  0.005846  0.005907 -2763.845312 -2637.998418
362  0.243353 -2524.130597  0.242880  0.243689 -2637.998418 -2508.562989
185  0.116671 -2492.995381  0.116419  0.117566 -2508.562989 -2460.600593
116  0.045538 -2428.205805  0.044764  0.045538 -2460.600593 -2341.403275
123  0.043663 -2254.600745  0.042534  0.043827 -2341.403275 -2180.293411
181  0.084314 -2105.986077  0.084267  0.085344 -2180.293411 -2102.637763
183  0.066123 -2099.289449  0.066081  0.066175 -2102.637763 -1988.599853
602  0.445063 -1877.910257  0.444075  0.445118 -1988.599853 -1810.654197
83   0.063649 -1743.398136  0.063450  0.063739 -181

In [236]:
def nu_f(z, r2D, nu_0, k):
    r2D = r2D+1
    return nu_0*z*np.power((np.power(r2D,2) + z**2),-k/2.)*np.power((z**2/np.power(r2D,2) +1), k/2.)* \
           hyp2f1(0.5, k/2., 1.5, -(z**2/np.power(r2D,2)))

def nu_f_2(r2D, nu_0, k):
    return nu_f(10., r2D, nu_0, k)-nu_f(-10., r2D, nu_0, k)
#print nu_f(0., np.array(data.radius2D),3025., -3. )

def sig_f(z, r2D, sig_0, m):
    r2D = r2D+1
    nu_0 = sig_0 # change this later
    k = m
    return nu_0*z*np.power((np.power(r2D,2) + z**2),-k/2.)*np.power((z**2/np.power(r2D,2) +1), k/2.)* \
           hyp2f1(0.5, k/2., 1.5, -(z**2/np.power(r2D,2)))

def sig_f_2(r2D, nu_0, k):
    return sig_f(10., r2D, nu_0, k)-nu_f(-10., r2D, nu_0, k)


def vel_spread(v, sigma2D, v_mean, Ndr):
    return Ndr/(np.sqrt(2.*np.pi*sigma2D))*np.exp(-np.power((v-v_mean),2)/(2*np.power(sigma2D,2)))
                                                   
    

def log_likelihood(sig_0, m, nu_0, k, v_mean=52.580055):
    # note that everything here is performed only on cells that actually have galaxies in them
    
    #N_r = quad(nu_f, np.array(data.dr_in), np.array(data.dr_out), args=(np.array(data.radius2D), nu_0, k))#scarry integral   #number of galaxies per the dr range
    N_r = nu_f_2(np.array(data.radius2D), nu_0, k)
    print N_r
    #sigma_2D = quad(sig_f, data.dr_in, data.dr_out, args=(data.radius2D, sig_0, m))# another scarry integral # projected sigma in that grid cell
    sigma_2D = sig_f_2(np.array(data.radius2D),  sig_0, m)
    #N_v =  quad(vel_spread, data.dv_in, data.dv_out, args=(sigma2D, v_mean, N_r))# integral from dv1 to dv2 of gaussian with sigma 2d, that is normalized by N_r # number of galaxies in the dv range
    N_v = vel_spread(np.array(data.v), sigma_2D, v_mean, N_r)
    N_det = N_r*N_v
    print "all detections:", np.sum(N_det)
    return (-1.*np.sum(N_det)*(np.sum(np.log(N_det))))

    
def log_prior(m, b):
    return 0.
    
def straight_line_log_posterior(x,y,sigmay, m,b):
    return (log_likelihood(x,y,sigmay, m,b) + log_prior(m, b))
# initial parameters (from actual distribution used)
nu_0_th = 3025.
k_th = -3.
sig_0_th = 1000.
m_th = -0.2 
v_mean =52.580055# taken from the data
log_likelihood(sig_0_th, m_th, nu_0_th, k_th)

[ 15682537.05951158  15588222.20907995  15619375.84037645
  15710955.51830066  15592812.65142811  15845600.21158316
  15703633.41373123  15631053.18409885  15629208.91177915
  15669987.04821154  15651534.44463486  16105955.25236379
  15649050.63412499  15594443.40849339  15854106.29697278
  15875229.93487711  15626526.40634586  15729843.39570693
  25691122.02088057  15686297.43220005  15658156.34802771
  15594474.87951704  17180757.7021298   15642542.59654254
  16161442.26650042  15847110.5039012   15679740.96936247
  15599617.63055013  15933168.55921624  15717900.80159044
  15710934.07393329  15730212.45864168  16164943.62154482
  15605698.37331127  15632689.64272904  16094160.73206457
  15754967.69528737  16116113.79505699  15700412.96457721  19408146.0246111
  15654231.23122854  15642239.73384584  15720401.1606667   15901898.26281636
  15916715.6178193   15732802.2260445   15650249.44445317
  15646007.34516187  16362041.33051939  17200543.35611482
  16085511.23128734  15800506.36550

-2.2123430433898848e+19

all detections: 9.70403744112e+16


-3.1066257907511145e+21